In [ ]:
!nvidia-smi

Mon Feb  9 15:04:56 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!pip uninstall -y torch torchvision torchaudio mamba-ssm causal-conv1d

!pip install torch==2.7.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

!pip install https://github.com/Dao-AILab/causal-conv1d/releases/download/v1.6.0/causal_conv1d-1.6.0+cu12torch2.7cxx11abiFALSE-cp312-cp312-linux_x86_64.whl

Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cu126
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 74.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 100.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 MB 77.8 MB/s eta 0:00:0000:0100:01
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requ

In [1]:
!pip install https://github.com/state-spaces/mamba/releases/download/v2.3.0/mamba_ssm-2.3.0+cu12torch2.7cxx11abiFALSE-cp312-cp312-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.3/533.3 MB 2.5 MB/s eta 0:00:0000:0100:01


In [4]:
import os
from google.colab import drive

# 1. Mount the Drive (The boring part)
drive.mount('/content/drive')

# 2. The Magic Shortcut ✨
# This makes your deep folder appear right at /content/project
my_drive_path = '/content/drive/MyDrive/SKRIPSIAHMOMENT_colab'
shortcut_path = '/content/SKRIPSIAHMOMENT_colab'

if not os.path.exists(shortcut_path):
    os.symlink(my_drive_path, shortcut_path)
    print(f"🔗 Linked! Your files are now at: {shortcut_path}")

# 3. Enter the zone
%cd {shortcut_path}
!ls  # Verify you see your files

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/SKRIPSIAHMOMENT_colab
mamba_quin  notebooks+pdf  README.md  sync_to_colab.sh	utils


In [2]:
!git clone https://github.com/Taya-san/SKRIPSIAHMOMENT.git

Cloning into 'SKRIPSIAHMOMENT'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 161 (delta 97), reused 111 (delta 47), pack-reused 0 (from 0)
Receiving objects: 100% (161/161), 820.41 KiB | 3.48 MiB/s, done.
Resolving deltas: 100% (97/97), done.


In [3]:
%cd SKRIPSIAHMOMENT/
%ls
!git pull

/content/SKRIPSIAHMOMENT
mamba_quin/  notebooks+pdf/  README.md  utils/
Already up to date.


In [2]:
import torch
from torch import nn
import numpy as np
import transformers
from transformers import AutoModel, AutoConfig, AutoTokenizer
from datasets import load_dataset
import sklearn.model_selection
import mamba_ssm
try:
    from mamba_ssm.ops.triton.selective_state_update import selective_state_update
    from mamba_ssm.ops.selective_scan_interface import selective_scan_fn, mamba_inner_fn
    
    mamba_ssm.selective_state_update = selective_state_update
    mamba_ssm.selective_scan_fn = selective_scan_fn
    mamba_ssm.mamba_inner_fn = mamba_inner_fn

except ImportError as e:
    print(f"Patch failed! Error: {e}")

In [11]:
from mamba_quin import MambaQuin, MambaQuinConfig
from utils import train_modelnoclt

In [ ]:
backbone_cfg = AutoConfig.from_pretrained("state-spaces/mamba-130m-hf")
pretrained_backbone = AutoModel.from_pretrained("state-spaces/mamba-130m-hf")

config = MambaQuinConfig(
    backbone_config = backbone_cfg.to_dict(),
    num_classes = 2
)

model = AutoModel.from_config(config)
model.backbone.load_state_dict(pretrained_backbone.state_dict())

Loading weights:   0%|          | 0/242 [00:00<?, ?it/s]

In [13]:
dataset = load_dataset("stanfordnlp/imdb")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [14]:
tokenizer = AutoTokenizer.from_pretrained(config.backbone_model)

In [15]:
train_modelnoclt(model, tokenizer, 3, dataset_dict=dataset, batch_size=4, tr_data=None, ts_data=None)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Epoch 1/3: 100%|██████████| 6250/6250 [08:22<00:00, 12.43it/s, loss=0.77] 


Epoch 1 loss: 1.0542


Epoch 2/3: 100%|██████████| 6250/6250 [08:24<00:00, 12.39it/s, loss=0.156] 


Epoch 2 loss: 0.3034


Epoch 3/3: 100%|██████████| 6250/6250 [08:24<00:00, 12.38it/s, loss=0.24]  

Epoch 3 loss: 0.2786


In [ ]:
from huggingface_hub import login
login(token = "", add_to_git_credential=True)

In [31]:
model.push_to_hub("GiganticLemon/mambaquin-small")
tokenizer.push_to_hub("GiganticLemon/mambaquin-small")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...03ndvr9/model.safetensors:   6%|6         | 33.5MB /  520MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/GiganticLemon/mambaquin-small/commit/fcd4f188bafc6924497b398a155cd9fff7f6f223', commit_message='Upload tokenizer', commit_description='', oid='fcd4f188bafc6924497b398a155cd9fff7f6f223', pr_url=None, repo_url=RepoUrl('https://huggingface.co/GiganticLemon/mambaquin-small', endpoint='https://huggingface.co', repo_type='model', repo_id='GiganticLemon/mambaquin-small'), pr_revision=None, pr_num=None)

In [32]:
model_test = AutoModel.from_pretrained("GiganticLemon/mambaquin-small")
tokenizer_test = AutoModel.from_pretrained("GiganticLemon/mambaquin-small")

config.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/520M [00:00<?, ?B/s]

RuntimeError: You are using `from_pretrained` with a meta device context manager or `torch.set_default_device('meta')`.
This is an anti-pattern as `from_pretrained` wants to load existing weights.
If you want to initialize an empty model on the meta device, use the context manager or global device with `from_config`, or `ModelClass(config)`

In [10]:
!nvidia-smi

Mon Feb  9 16:01:57 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----